# Run memento for Perturb-seq benchmarking


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2


In [3]:
import sys
sys.path.append('/home/ssm-user/Github/memento')

In [4]:
import scanpy as sc
import scipy.stats as stats
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import string
import random
import logging
import itertools
import statsmodels.api as sm
import seaborn as sns
import random

In [5]:
import memento

In [6]:
import memento.model.rna as rna
import memento.estimator.hypergeometric as hg

In [7]:
data_path = '/data_volume/memento/'

In [8]:
logging.basicConfig(
    format="%(asctime)s %(process)-7s %(levelname)-8s %(message)s",
    level=logging.INFO, 
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.captureWarnings(True)

### Perturbseq

In [9]:
adata = sc.read(data_path + 'tfko140/tfko.sng.guides.full.ct.details.h5ad')

Only considering the two last: ['.details', '.h5ad'].
Only considering the two last: ['.details', '.h5ad'].


In [10]:
donors = adata.obs.donor.drop_duplicates().tolist()

In [11]:
guide_pairs = pd.read_csv(data_path + 'tfko140/benchmarking/guide_pairs_to_test.csv')

In [12]:
rna.MementoRNA.setup_anndata(
    adata=adata,
    q_column='q',
    label_columns=['donor', 'guide'])

2023-07-14 10:57:10 2047692 INFO     setup_anndata: creating groups
2023-07-14 10:57:35 2047692 INFO     setup_anndata: computing cell sizes


In [13]:
# Define expressed genes
adata.var['expr_genes'] = (adata.X.mean(axis=0).A1 > 0.02)
adata = adata[:, adata.var['expr_genes']]

In [15]:
model = rna.MementoRNA(adata=adata)

In [ ]:
model.compute_estimate(
estimand='mean',
get_se=True,
n_jobs=30,
)

In [ ]:
### QUASI GLM

%%time
for idx, row in guide_pairs.iterrows():
    
    print(idx)
    sets = [(row['target1_guide1'], row['target2_guide1']), (row['target1_guide2'], row['target2_guide2'])]
    for t1, t2 in sets:
        

        groups = [f'memento_group^{d}^{g}' for d,g in itertools.product(donors, [t1,t2])]
        treatment = pd.DataFrame(index=groups)
        covariate = pd.DataFrame(index=groups)
        treatment['is_target1'] = treatment.index.str.contains(t1).astype(float)
        covariate['donor'] = covariate.index.str.split('^').str[1]
        covariate = pd.get_dummies(covariate, drop_first=True).astype(float)
        covariate = sm.add_constant(covariate)
        
        result = model.differential_mean(covariates=covariate, treatments=treatment, family='quasiGLM', n_jobs=30, verbose=2)
        result.to_csv(data_path + f'tfko140/benchmarking/memento_results/{t1}_{t2}.csv')

In [ ]:
%%time

### WLS
for idx, row in guide_pairs.iterrows():
    
    print(idx)
    sets = [(row['target1_guide1'], row['target2_guide1']), (row['target1_guide2'], row['target2_guide2'])]
    for t1, t2 in sets:
        

        groups = [f'memento_group^{d}^{g}' for d,g in itertools.product(donors, [t1,t2])]
        treatment = pd.DataFrame(index=groups)
        covariate = pd.DataFrame(index=groups)
        treatment['is_target1'] = treatment.index.str.contains(t1).astype(float)
        covariate['donor'] = covariate.index.str.split('^').str[1]
        covariate = pd.get_dummies(covariate, drop_first=True).astype(float)
        covariate = sm.add_constant(covariate)
        
        result = model.differential_mean(covariates=covariate, treatments=treatment, estimator='log_mean', family='WLS', n_jobs=1, verbose=2)
        result.to_csv(data_path + f'tfko140/benchmarking/memento_results/{t1}_{t2}_WLS.csv')

0


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.2s finished


1


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.2s finished


2


...

95


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.2s finished


96


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 9133 out of 9133 | elapsed:    7.5s finished


97
